给出判断指标，权重以及指标类型

In [1]:
import numpy as np
import pandas as pd

# 三个方案：A/B/C
# 四个指标：成本(Cost)、效率(Eff)、可靠性(Reliability)、寿命(Life)

data = pd.DataFrame({
    "Cost": [250, 200, 300],         # 成本型指标
    "Efficiency": [0.80, 0.60, 0.90],
    "Reliability": [0.95, 0.90, 0.85],
    "Life": [10, 8, 12]
}, index=["A", "B", "C"])

# 权重（归一化）
weights = np.array([0.2, 0.3, 0.3, 0.2])

# 指标类型：1=效益型，-1=成本型
types = np.array([-1, 1, 1, 1])

print("原始数据：")
display(data)


原始数据：


,Cost,Efficiency,Reliability,Life
A,250,0.8,0.95,10
B,200,0.6,0.90,8
C,300,0.9,0.85,12


正向化

In [2]:
normalized_data = data.copy()

for j in range(data.shape[1]):
    if types[j] == -1:  # 成本型 → 越小越好 → 转换
        normalized_data.iloc[:, j] = data.iloc[:, j].max() - data.iloc[:, j]
    else:
        normalized_data.iloc[:, j] = data.iloc[:, j]

print("\n正向化后数据：")
display(normalized_data)



正向化后数据：


,Cost,Efficiency,Reliability,Life
A,50,0.8,0.95,10
B,100,0.6,0.90,8
C,0,0.9,0.85,12


归一化

In [3]:
z = normalized_data / np.sqrt((normalized_data ** 2).sum())

print("\n归一化矩阵 Z：")
display(z)



归一化矩阵 Z：


,Cost,Efficiency,Reliability,Life
A,0.447214,0.594635,0.608799,0.569803
B,0.894427,0.445976,0.576757,0.455842
C,0.000000,0.668965,0.544715,0.683763


加权矩阵

In [4]:
V = z * weights

print("\n加权矩阵 V：")
display(V)



加权矩阵 V：


,Cost,Efficiency,Reliability,Life
A,0.089443,0.178391,0.182640,0.113961
B,0.178885,0.133793,0.173027,0.091168
C,0.000000,0.200689,0.163415,0.136753


求正负最值

In [5]:
v_plus = V.max()
v_minus = V.min()

print("\n正理想解 v+：")
display(pd.DataFrame(v_plus).T)

print("\n负理想解 v-：")
display(pd.DataFrame(v_minus).T)



正理想解 v+：


,Cost,Efficiency,Reliability,Life
0,0.178885,0.200689,0.18264,0.136753



负理想解 v-：


,Cost,Efficiency,Reliability,Life
0,0.0,0.133793,0.163415,0.091168


计算距离

In [6]:
D_plus = np.sqrt(((V - v_plus) ** 2).sum(axis=1))
D_minus = np.sqrt(((V - v_minus) ** 2).sum(axis=1))

distances = pd.DataFrame({"D+": D_plus, "D-": D_minus})
print("\n各方案到正负理想解的距离：")
display(distances)



各方案到正负理想解的距离：


,D+,D-
A,0.094956,0.104298
B,0.081520,0.179144
C,0.179916,0.080951


最终判断指标计算

In [7]:
C = D_minus / (D_plus + D_minus)

result = pd.DataFrame({"Ci": C, "Rank": C.rank(ascending=False)})

print("\nTOPSIS 最终评分与排名：")
display(result.sort_values("Rank"))



TOPSIS 最终评分与排名：


,Ci,Rank
B,0.687260,1.0
A,0.523441,2.0
C,0.310316,3.0
